In [2]:
import  os
os.chdir('../')
%pwd

'/run/media/suraj/1366967307359354/project/KidneyClassification/Kideney_desease_classification'

In [3]:


os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/SurajKumar2235/Kideney_disease_classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="SurajKumar2235"  
os.environ["MLFLOW_TRACKING_PASSWORD"]="7cefaf8f22132447930396e29063f8aa77e2aa25"


In [4]:
import tensorflow as tf
from keras.metrics import MeanAbsolutePercentageError  


2024-03-27 17:37:11.642048: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-27 17:37:11.653272: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-27 17:37:11.661063: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-27 17:37:11.752171: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-27 17:37:15.903542: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [5]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.keras")


In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass (frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories,save_json


In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.keras",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/SurajKumar2235/Kideney_disease_classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [9]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [10]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        # scores = {"loss": self.score[0], "accuracy": self.score[1]}
        scores={'loss':self.score}
        
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                # {"loss": self.score[0], "accuracy": self.score[1]}
                {'loss':self.score}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [11]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-03-27 17:37:32,399:INFO:common: yaml file: config/config.yaml loaded successfully]
[2024-03-27 17:37:32,411:INFO:common: yaml file: params.yaml loaded successfully]
[2024-03-27 17:37:32,415:INFO:common: created directory at: artifacts]


Found 139 images belonging to 2 classes.


/run/media/suraj/1366967307359354/project/KidneyClassification/Kideney_desease_classification/kidney/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 31s 3s/step - loss: 0.7855
[2024-03-27 17:38:05,527:INFO:common: json file saved at: scores.json]


2024/03/27 17:38:09 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


[2024-03-27 17:48:37,381:WARNING:connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'TimeoutError('The write operation timed out')': /SurajKumar2235/Kideney_disease_classification.mlflow/api/2.0/mlflow-artifacts/artifacts/4da2f174c21747948310eced9b1dffa5/f7e4267cd7864228ab1d07542245d1bb/artifacts/model/data/model.keras]


Successfully registered model 'VGG16Model'.
2024/03/27 18:04:56 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
